## Download model

It's important that your model have this directory structure for Triton Inference Server to be able to load it. [Read more about the directory structure that Triton expects](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/model_repository.html).

In [1]:
!pip install azure-storage-blob

In [2]:
import os
import sys
from pathlib import Path
from src.model_utils import download_triton_models, delete_triton_models

prefix = Path(".")
download_triton_models(prefix)

successfully downloaded model: densenet_onnx


## Register model

The registered model should follow the Triton specified model folder structure for Triton Inference Server to be able to load it.

In [10]:
subscripton = "92c76a2f-0e1c-4216-b65e-abf7a3f34c1e"
resource_group = "gsc-aml-lite-rg2"
workspace = "gsc-aml-ws3"
model_name = "densenet-onnx"
endpoint_name = "arz12"

In [11]:
!az ml model create -n $model_name -v 1 -l models -g $resource_group -w $workspace --subscription $subscripton

Command group 'ml model' is experimental and under development. Reference and support levels: https://aka.ms/CLI_refstatus
{
  "creation_context": {
    "created_at": "2021-05-19T08:34:35.392674+00:00",
    "created_by": "Arzoo Aneja (ZEN3 INFOSOLUTIONS AMERICA INC)",
    "created_by_type": "User",
    "last_modified_at": "2021-05-20T08:06:12.465692+00:00",
    "last_modified_by": "Arzoo Aneja (ZEN3 INFOSOLUTIONS AMERICA INC)",
    "last_modified_by_type": "User"
  },
  "datastore": "azureml:/subscriptions/92c76a2f-0e1c-4216-b65e-abf7a3f34c1e/resourceGroups/gsc-aml-lite-rg2/providers/Microsoft.MachineLearningServices/workspaces/gsc-aml-ws3/datastores/workspaceblobstore",
  "flavors": {},
  "id": "azureml:/subscriptions/92c76a2f-0e1c-4216-b65e-abf7a3f34c1e/resourceGroups/gsc-aml-lite-rg2/providers/Microsoft.MachineLearningServices/workspaces/gsc-aml-ws3/models/densenet-onnx/versions/1",
  "name": "densenet-onnx",
  "path": "az-ml-artifacts/57396fa60078750da5128ce974a4a65f/models",
  "pr

In [12]:
!az ml model show -n $model_name -v 1 -g $resource_group -w $workspace --subscription $subscripton

Command group 'ml model' is experimental and under development. Reference and support levels: https://aka.ms/CLI_refstatus
{
  "creation_context": {
    "created_at": "2021-05-19T08:34:35.392674+00:00",
    "created_by": "Arzoo Aneja (ZEN3 INFOSOLUTIONS AMERICA INC)",
    "created_by_type": "User",
    "last_modified_at": "2021-05-20T08:06:12.465692+00:00",
    "last_modified_by": "Arzoo Aneja (ZEN3 INFOSOLUTIONS AMERICA INC)",
    "last_modified_by_type": "User"
  },
  "datastore": "azureml:/subscriptions/92c76a2f-0e1c-4216-b65e-abf7a3f34c1e/resourceGroups/gsc-aml-lite-rg2/providers/Microsoft.MachineLearningServices/workspaces/gsc-aml-ws3/datastores/workspaceblobstore",
  "flavors": {},
  "id": "azureml:/subscriptions/92c76a2f-0e1c-4216-b65e-abf7a3f34c1e/resourceGroups/gsc-aml-lite-rg2/providers/Microsoft.MachineLearningServices/workspaces/gsc-aml-ws3/models/densenet-onnx/versions/1",
  "name": "densenet-onnx",
  "path": "az-ml-artifacts/57396fa60078750da5128ce974a4a65f/models",
  "pr

## Create endpoint

Deploy to a pre created AKS compute

In [13]:
!az ml endpoint create -g $resource_group -w $workspace -n $endpoint_name -f endpoint-mir-gpu-copy.yml

Command group 'ml endpoint' is experimental and under development. Reference and support levels: https://aka.ms/CLI_refstatus

The deployment request gsc-aml-ws3-arz12-7184531 was accepted,  status can be found in the link below: 
https://ms.portal.azure.com/#blade/HubsExtension/DeploymentDetailsBlade/overview/id/%2Fsubscriptions%2F92c76a2f-0e1c-4216-b65e-abf7a3f34c1e%2FresourceGroups%2Fgsc-aml-lite-rg2%2Fproviders%2FMicrosoft.Resources%2Fdeployments%2Fgsc-aml-ws3-arz12-7184531

Registering code version (a2cd405d-9214-4f41-902e-956c6bff421d:1)  Done (17s)
Registering environment version (single-env:1)  Done (24s)
Creating endpoint arz12 ....  Done (1m 7s)
Creating deployment etblue: Failed with operation id= 9FC29A16C1722066, service request id=1d8e2c5b-ab05-44a7-8d25-4dc0b4ea1498, status=BadRequest, error message = {'additional_properties': {}, 'code': None, 'message': None, 'target': None, 'details': None, 'additional_info': None}.
More details: None
Polling hit the exception (Deploy

In [ ]:
!az ml endpoint show -g $resource_group -w $workspace -n $endpoint_name

## Test Webservice

Get scoring URI and auth token

In [ ]:
!az ml endpoint show -n $endpoint_name --query "scoring_uri"

In [ ]:
!az ml endpoint list-keys -g $resource_group -w $workspace -n $endpoint_name

In [ ]:
import requests

service_key = "service_key"
headers = {}
headers["Authorization"] = f"Bearer {service_key}"

# Check the state of server.
service_url = "service_url"
requests.get(f"{service_url}/v2/health/ready", headers=headers)

In [ ]:
# Check the status of model.
resp = requests.post(f"{service_url}/v2/repository/index", headers=headers)
print(resp.text)

In [ ]:
# Check metadata of model for inference 
resp = requests.get(f"{service_url}/v2/models/densenet_onnx", headers=headers)
print(resp.text)